In [ ]:
import csv
import json
import sys

csv.field_size_limit(sys.maxsize)

from pathlib import Path

In [ ]:
# submission = "packages_RSNA_20211117" # done
# submission = "packages_RSNA_20211214" # partially done, indexed
# submission = "packages_RSNA_20220105" # partially done, indexed
# submission = "packages_RSNA_20220114" # partially done, indexed
# submission = "packages_RSNA_20220124" # partially done, indexed
# submission = "packages_RSNA_20220211" # partially done, indexed
submission = "packages_RSNA_20220214" # partially done, indexed
# submission = "packages_midrc-ricord-2021-08-20" # todo
# submission = "packages_midrc-ricord-2021-09-02" # done
# submission = "packages_midrc-ricord-2021-09-22" # done
# submission = "packages_midrc-ricord-2021-10-06" # done
# submission = "packages_midrc-ricord-2021-10-26" # done
submission_path = Path("/Users/andrewprokhorenkov/CTDS/projects/midrc/midrc_indexing_scripts/packages_rsna")
submission_path = submission_path / submission
package_files = submission_path / "packages"
sequestration_master_file_path = "/Users/andrewprokhorenkov/CTDS/projects/midrc/midrc_indexing_scripts/sequestration/master_sequestration_locations_10162_2022-03-18.tsv"

In [ ]:
seq_master = {}
with open(sequestration_master_file_path) as sequestration_master_file:
    reader = csv.DictReader(sequestration_master_file, delimiter="\t")

    for row in reader:
        seq_master[row["case_ids"]] = row["dataset"]

In [ ]:
open_packages = []
seq_packages = []
missing_packages = []

for package_filepath in package_files.iterdir():
    with open(package_filepath) as package_file:
        reader = csv.DictReader(package_file, delimiter="\t")
        for row in reader:
            item = row

            file_name = item["file_name"]
            case_id = file_name.split("/")[0]

            package_contents = json.loads(item["package_contents"].replace("'", "\""))
            for p in package_contents:
                p["size"] = int(p["size"])
            
            item["package_contents"] = json.dumps(package_contents)

            # if case_id not in seq_master:
            #     continue

            dataset = seq_master.get(case_id, None)

            if dataset == "Open":
                bucket = "s3://open-data-midrc/"
                authz = json.dumps(["/programs/Open/projects/R1"])
            elif dataset == "Seq":
                bucket = "s3://sequestered-data-midrc/"
                authz = json.dumps(["/programs/SEQ_Open/projects/R3"])
            else:
                authz = ""
                bucket = ""

            item["authz"] = authz
            item["url"] = f"{bucket}{item['url']}"

            if dataset == "Open":
                open_packages.append(item)
            elif dataset == "Seq":
                seq_packages.append(item)
            else:
                missing_packages.append(item)

In [ ]:
datasets = [
    (f"packages_open_{submission.removeprefix('packages_')}.tsv", open_packages),
    (f"packages_seq_{submission.removeprefix('packages_')}.tsv", seq_packages),
    (f"packages_missing_{submission.removeprefix('packages_')}.tsv", missing_packages),
]

fieldnames = [
    "record_type",
    "guid",
    "md5",
    "size",
    "authz",
    "url",
    "file_name",
    "package_contents",
]

for filename, dataset in datasets:
    with open(
        submission_path / filename,
        "w",
    ) as f:
        writer = csv.DictWriter(f, delimiter="\t", fieldnames=fieldnames)
        writer.writeheader()

        for item in dataset:
            writer.writerow(item)